<a href="https://colab.research.google.com/github/alejandro-cermeno/2021_Market_Timing-Cermeno/blob/main/resultados_excel_latex.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# **1.**

In [182]:
%%capture

# Instalacion

!pip install arch


# Librerias

import math
import numpy as np
import pandas as pd
import arch
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.metrics import mean_squared_error


# Obtencion de datos

ruta = 'https://git.io/Jnovh'

precios = pd.read_excel(ruta, sheet_name = 'source', index_col = 0)
retornos = price2ret(precios)


# Etiquetas

mercado = ['(MERVAL)', '(IBOV)', '(IPSA)', '(IGBC)', '(MEXBOL)', '(SPBLPGPTR)']
pais    = ['Argentina', 'Brazil', 'Chile', 'Colombia', 'Mexico', 'Peru']


# Especificaciones

  # Media

media = ['Zero', 'Constant', 'AR']
media_nom = ['Cero', 'Constante', 'AR']

  # Varianza

arch_params = {'vol': 'ARCH'}
garch_params = {'p':1, 'q':1, 'vol':'GARCH'}
grj_params = {'p':1, 'o':1, 'q':1, 'vol':'GARCH'}
egarch_params = {'p': 1, 'q': 1, 'o': 1, 'vol': 'EGARCH'}

varianza = [arch_params, garch_params, grj_params, egarch_params]
varianza_nom = ['ARCH', 'GARCH', 'GRJ', 'EGARCH']

  # Distribuciones

dist = ['normal', 't', 'skewt']
dist_nom = ['N', 't', 'sk'] 


# Almacenamiento de resultados

tabla_resultados = pd.DataFrame()

# **2)**

In [183]:
def mdl_volatilidad(
    ts, media, varianza, dist, media_nom, varianza_nom, dist_nom):

   # Especificacion
  mdl = arch.arch_model(ts, mean = media, **varianza, dist = dist)
  
  # Estimacion
  mdl = mdl.fit(disp='off')

  # Resultados

  # Media

  media_nom = pd.Series(media_nom, index = ['Media'])

  # Varianza

  varianza_nom = pd.Series(varianza_nom, index = ['Varianza'])

  # Distribucion

  dist_nom = pd.Series(dist_nom, index = ['Distribucion'])

  # P-values

  pvalues = mdl.pvalues

  # Coeficientes

  coef = round( mdl.params, 3 )

  # Log-verosimilutud

  loglik = round( mdl.loglikelihood, 3 )
  loglik = pd.Series(loglik, index = ['log-lik'])

  # Valor del criterio de informacion

  bic = round( mdl.bic, 3 )
  bic = pd.Series(bic, index = ['BIC'])

  # Volatilidad proyectada y observda

  vol_proyectada = mdl.conditional_volatility ** 2
  vol_observada = ts ** 2

  # Raíz del error cuadratico medio (RMSE)

  rmse = math.sqrt( mean_squared_error(vol_observada, vol_proyectada) )
  mrse = round( rmse, 3 )
  rmse = pd.Series(rmse, index = ['RMSE'])

  # Tabla resultado

  resultado = pd.concat([media_nom, varianza_nom, dist_nom, coef, bic,
                             loglik, rmse]) 
  resultado = pd.DataFrame(resultado).T

  return resultado


################################################################


def price2ret(price):

  '''

  Convierte los datos contenidos en la serie en price de niveles a retornos.
  Requiere numpy.

  Entradas:

    price: (array_like) Serie de precios. 

  '''
  
  ret = (100 * (np.log(price) - np.log(price.shift(1))))
  
  return ret

## **3.**

In [184]:
%%capture

numero_series = retornos.shape[1]

for i in range(0, numero_series):   # Serie

  ts = retornos.iloc[:, i].dropna()

  for j in range(len(media)):       # Especificacion media

    for k in range(len(varianza)):  # Especificacion varianza

      for l in range(len(dist)):    # Especificacion distribucion
        
        # Se fija la cantidad de decimales
        pd.set_option('precision', 3)
        
        resultado = mdl_volatilidad(ts, media[j], varianza[k], dist[l], media_nom[j], varianza_nom[k], dist_nom[l])
        tabla_resultados = tabla_resultados.append(resultado, sort=False)

  encabezado_pais = {'Media': [pais[i]], 'Varianza': [mercado[i]]}
  encabezado_pais = pd.DataFrame.from_dict(encabezado_pais)
  tabla_resultados = pd.concat([encabezado_pais, tabla_resultados])

# **4)**

In [186]:
# Modificaciones a la tabla 

  # Remplazar NaN por espacios en blanco

tabla_resultados = tabla_resultados.fillna('')

  # Reiniciar el index

tabla_resultados = tabla_resultados.reset_index(drop=True)

  # Ordenar las columnas
  
nombres_columnas = list(tabla_resultados.columns)

columnas_derecha =  ['BIC',  'log-lik', 'RMSE']
columnas_izquierda = [l for l in nombres_columnas if l not in columnas_derecha]

orden_columnas = columnas_izquierda + columnas_derecha

tabla_resultados = tabla_resultados[orden_columnas]


# Exportar resultados

  # Excel

tabla_resultados.to_excel('resultados_excel.xlsx')  

  # LaTeX

latex = tabla_resultados.to_latex()

#adicionar = '\resizebox{1\textwidth}{!}{\begin{centering}'

with open("resultados_latex.tex", "w") as tex:
    tex.write(latex)

# **EN DESARROLLO**

In [187]:
# Coeficientes con significancia (a, b, c)
#coefs_con_significancia = pd.Series(colocar_letra_significancia(coef, 
#                                                                pvalues))
#coefs_con_significancia = pd.Series(data=coefs_con_significancia.values, 
#                                    index = coef.index)


def colocar_letra_significancia(coef, pvalues):

  #######################################
  # PENDIENTE:                          # 
  # * Documentar                        #
  # * Paso previo de verificacion       #
  #######################################

  nivel_significancia_letra = nivel_significancia_letras(pvalues)

  coefs_con_significancia = []

  for i in range(len(coef)):

    _coef_con_significancia_ = ('$ $' + str(coef[i]) + '^{'+ str(nivel_significancia_letra[i]) +'}$ $')

    coefs_con_significancia.append(str(_coef_con_significancia_))

  return coefs_con_significancia



def nivel_significancia_letras(pvalues):

  #######################################
  # PENDIENTE:                          # 
  # * Documentar                        #
  #######################################

  ''' 
  pvalues:  (array_like)

  '''

  nivel_significancia_letras = pd.Series()

  # Clasificacion del nivel de significancia
  for i in range(len(pvalues)):

    if pvalues[i] <= 0.01:
      letra = 'a'                     # a: significancia 1%
    elif pvalues[i] <= 0.05:                
      letra = 'b'                     # b: significancia 5%
    elif pvalues[i] <= 0.1:
      letra = 'c'                     # c: significancia 10%
    else:
      letra = ''                           #    no significativo 

    letra = pd.Series(letra, index = [pvalues.index[i]])
    nivel_significancia_letras = nivel_significancia_letras.append(letra)

  nivel_significancia_letras = nivel_significancia_letras.fillna('')

  return nivel_significancia_letras